## SQL at Scale with Spark SQL

Welcome to the SQL mini project. For this project, you will use the Databricks Platform and work through a series of exercises using Spark SQL. The dataset size may not be too big but the intent here is to familiarize yourself with the Spark SQL interface which scales easily to huge datasets, without you having to worry about changing your SQL queries. 

The data you need is present in the mini-project folder in the form of three CSV files. This data will be imported in Databricks to create the following tables under the __`country_club`__ database.

<br>
1. The __`bookings`__ table,
2. The __`facilities`__ table, and
3. The __`members`__ table.

You will be uploading these datasets shortly into the Databricks platform to understand how to create a database within minutes! Once the database and the tables are populated, you will be focusing on the mini-project questions.

In the mini project, you'll be asked a series of questions. You can solve them using the databricks platform, but for the final deliverable,
please download this notebook as an IPython notebook (__`File -> Export -> IPython Notebook`__) and upload it to your GitHub.

### Creating the Database

We will first create our database in which we will be creating our three tables of interest

In [0]:
%sql 
drop database if exists country_club cascade;
create database country_club;
show databases;

namespace
country_club
default


### Creating the Tables

In this section, we will be creating the three tables of interest and populate them with the data from the CSV files already available to you. 
To get started, first upload the three CSV files to the DBFS as depicted in the following figure

![](https://i.imgur.com/QcCruBr.png)


Once you have done this, please remember to execute the following code to build the dataframes which will be saved as tables in our database

In [0]:
# File location and type
file_location_bookings = "/FileStore/tables/Bookings.csv"
file_location_facilities = "/FileStore/tables/Facilities.csv"
file_location_members = "/FileStore/tables/Members.csv"

file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
bookings_df = (spark.read.format(file_type) 
                    .option("inferSchema", infer_schema) 
                    .option("header", first_row_is_header) 
                    .option("sep", delimiter) 
                    .load(file_location_bookings))

facilities_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_facilities))

members_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_members))

### Viewing the dataframe schemas

We can take a look at the schemas of our potential tables to be written to our database soon

In [0]:
print('Bookings Schema')
bookings_df.printSchema()
print('Facilities Schema')
facilities_df.printSchema()
print('Members Schema')
members_df.printSchema()

Bookings Schema
root
-- bookid: integer (nullable = true)
-- facid: integer (nullable = true)
-- memid: integer (nullable = true)
-- starttime: string (nullable = true)
-- slots: integer (nullable = true)

Facilities Schema
root
-- facid: integer (nullable = true)
-- name: string (nullable = true)
-- membercost: double (nullable = true)
-- guestcost: double (nullable = true)
-- initialoutlay: integer (nullable = true)
-- monthlymaintenance: integer (nullable = true)

Members Schema
root
-- memid: integer (nullable = true)
-- surname: string (nullable = true)
-- firstname: string (nullable = true)
-- address: string (nullable = true)
-- zipcode: integer (nullable = true)
-- telephone: string (nullable = true)
-- recommendedby: integer (nullable = true)
-- joindate: string (nullable = true)

### Create permanent tables
We will be creating three permanent tables here in our __`country_club`__ database as we discussed previously with the following code

In [0]:
permanent_table_name_bookings = "country_club.Bookings"
bookings_df.write.format("parquet").saveAsTable(permanent_table_name_bookings)

permanent_table_name_facilities = "country_club.Facilities"
facilities_df.write.format("parquet").saveAsTable(permanent_table_name_facilities)

permanent_table_name_members = "country_club.Members"
members_df.write.format("parquet").saveAsTable(permanent_table_name_members)

### Refresh tables and check them

In [0]:
%sql
use country_club;
REFRESH table bookings;
REFRESH table facilities;
REFRESH table members;
show tables;

database,tableName,isTemporary
country_club,bookings,false
country_club,facilities,false
country_club,members,false


### Test a sample SQL query

__Note:__ You can use __`%sql`__ at the beginning of a cell and write SQL queries directly as seen in the following cell. Neat isn't it!

In [0]:
%sql
select MAX(membercost) from facilities 

max(membercost)
9.9


#### Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.

In [0]:
%sql
SELECT name 
FROM facilities
WHERE membercost != 0

name
Tennis Court 1
Tennis Court 2
Massage Room 1
Massage Room 2
Squash Court


####  Q2: How many facilities do not charge a fee to members?

In [0]:
%sql
SELECT COUNT(*) AS nofee_count
FROM facilities
WHERE membercost == 0

nofee_count
4


#### Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? 
#### Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [0]:
%sql
SELECT facid,
       name,
       membercost,
       monthlymaintenance
FROM facilities
WHERE (membercost != 0) AND (membercost < 0.2*monthlymaintenance)


facid,name,membercost,monthlymaintenance
0,Tennis Court 1,5.0,200
1,Tennis Court 2,5.0,200
4,Massage Room 1,9.9,3000
5,Massage Room 2,9.9,3000
6,Squash Court,3.5,80


#### Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

In [0]:
%sql
SELECT *
FROM facilities
WHERE facid IN (1, 5)

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
1,Tennis Court 2,5.0,25.0,8000,200
5,Massage Room 2,9.9,80.0,4000,3000


#### Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100? 
#### Return the name and monthly maintenance of the facilities in question.

In [0]:
%sql
SELECT monthlymaintenance,
       CASE WHEN monthlymaintenance > 100 THEN 'expensive' ELSE 'cheap' END AS facility_cost
FROM facilities

monthlymaintenance,facility_cost
200,expensive
200,expensive
50,cheap
10,cheap
3000,expensive
3000,expensive
80,cheap
15,cheap
15,cheap


#### Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution.

In [0]:
%sql
SELECT spark_catalog.country_club.members.firstname,
       spark_catalog.country_club.members.surname,
       joindate
FROM members
Where joindate = (SELECT MAX(joindate) FROM members)

firstname,surname,joindate
Darren,Smith,2012-09-26 18:08:45


####  Q7: How can you produce a list of all members who have used a tennis court?
- Include in your output the name of the court, and the name of the member formatted as a single column. 
- Ensure no duplicate data
- Also order by the member name.

In [0]:
%sql
-- will need to join facilities, bookings, and members
-- not supposed to do subquery yet
-- need to combine surname and firstname into a single column

SELECT DISTINCT CONCAT(firstname, ' ', surname) AS whole_name,
       name
FROM facilities
JOIN bookings 
    ON bookings.facid = facilities.facid
JOIN members
    ON members.memid = bookings.memid
WHERE facilities.name LIKE 'Tennis Court%'  
ORDER BY whole_name
-- if it is a different tennis court, is that still considered a duplicate? Can maybe do an INNER JOIN in that case


whole_name,name
Anne Baker,Tennis Court 1
Anne Baker,Tennis Court 2
Burton Tracy,Tennis Court 2
Burton Tracy,Tennis Court 1
Charles Owen,Tennis Court 2
Charles Owen,Tennis Court 1
Darren Smith,Tennis Court 2
David Farrell,Tennis Court 2
David Farrell,Tennis Court 1
David Jones,Tennis Court 2


#### Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? 

- Remember that guests have different costs to members (the listed costs are per half-hour 'slot')
- The guest user's ID is always 0. 

#### Include in your output the name of the facility, the name of the member formatted as a single column, and the cost.

- Order by descending cost, and do not use any subqueries.

In [0]:
%sql
-- specific day comes from bookings, cost comes from facilities, name of member comes from members
-- previous problem was successful joining bookings to facilities, then joining members. so order fac, book, mem
-- solution was having an issue with total_cost column... can do query to see that maximum member cost for any facility is 9.9
-- so should only see guests showing up. Had to repeat CASE statment due to order SQL executes things. 
SELECT facilities.name AS facility_name,
       CONCAT(members.firstname, ' ', members.surname) AS member_whole_name,
       CASE WHEN members.memid == 0 THEN guestcost ELSE membercost END AS total_cost 
FROM facilities
JOIN bookings 
    ON bookings.facid = facilities.facid
JOIN members
    ON members.memid = bookings.memid
WHERE (starttime LIKE '2012-09-14%') AND ((CASE WHEN members.memid == 0 THEN guestcost ELSE membercost END) > 30) 
ORDER BY 3 DESC
--limit 10

facility_name,member_whole_name,total_cost
Massage Room 1,GUEST GUEST,80.0
Massage Room 1,GUEST GUEST,80.0
Massage Room 1,GUEST GUEST,80.0
Massage Room 2,GUEST GUEST,80.0


#### Q9: This time, produce the same result as in Q8, but using a subquery.

In [0]:
%sql
--still want 2012-9-14, cost>30, output the name of the facility, the name of the member formatted as a single column, and the cost
SELECT t1.name AS facility_name,
       CONCAT(members.firstname, ' ', members.surname) AS member_whole_name,
       CASE WHEN members.memid == 0 THEN guestcost ELSE membercost END AS total_cost 
FROM members
JOIN (  SELECT * 
        FROM facilities
        JOIN bookings 
            ON bookings.facid = facilities.facid
        WHERE (starttime LIKE '2012-09-14%')) t1 
  ON t1.memid = members.memid
WHERE (starttime LIKE '2012-09-14%') AND ((CASE WHEN members.memid == 0 THEN guestcost ELSE membercost END) > 30) 
ORDER BY 3 DESC

facility_name,member_whole_name,total_cost
Massage Room 1,GUEST GUEST,80.0
Massage Room 1,GUEST GUEST,80.0
Massage Room 1,GUEST GUEST,80.0
Massage Room 2,GUEST GUEST,80.0


#### Q10: Produce a list of facilities with a total revenue less than 1000.
- The output should have facility name and total revenue, sorted by revenue. 
- Remember that there's a different cost for guests and members!

In [0]:
%sql
-- need to calculate the revenue for each of the 9 facilities, from the 4043 bookings. Slots is how many 30 min sessions.  
-- for every single bookings row, need to know price of facility (member or guest), and num of slots. multiply those two numbers together, and this should contribute a sum of income for that particular facility. so put that multiplication in a sum. The output of the sum will be
-- the subquery here finds name of facility, as well as the total revenue generated for that row. But with the group by facilities.name clause, we will find the sum of the revenues for all bookings for that particular facility! 
SELECT name, revenue FROM ( select facilities.name, sum(case when memid = 0 then slots * facilities.guestcost 
                                                       else slots * membercost END) as revenue
                            from bookings
                            inner join facilities
                              on bookings.facid = facilities.facid 
                            group by facilities.name)
WHERE (revenue < 1000)
ORDER BY revenue DESC

name,revenue
Pool Table,270.0
Snooker Table,240.0
Table Tennis,180.0
